In [14]:
# 导入需要的数据包
from sklearn.datasets import load_iris 
from sklearn.model_selection import train_test_split
from sklearn import tree
import pydotplus
from sklearn.externals.six import StringIO
from IPython.display import Image, display
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import talib
from jqdata import *

#load_iris是sklearn的测试数据，在这用来做决策树演示
iris = load_iris()

#建立最大深度为2的决策树，并用测试数据来训练这颗树


#开始利用机器学习算法计算，括号里面的n_estimators就是森林中包含的树的数目啦
clf_forest = RandomForestClassifier(n_estimators=50,max_depth=7)
clf_tree = tree.DecisionTreeClassifier(max_depth=7)

test_stock = '000001.XSHG'
start_date = datetime.date(2010, 6, 4)
end_date = datetime.date(2016, 6, 3)

trading_days = list(get_all_trade_days())
start_date_index = trading_days.index(start_date)
end_date_index = trading_days.index(end_date)

x_all = []
y_all = []

for index in range(start_date_index, end_date_index):
    # 得到计算指标的所有数据
    start_day = trading_days[index - 30]
    end_day = trading_days[index]
    stock_data = get_price(test_stock, start_date=start_day, end_date=end_day, frequency='daily', fields=['close'])
    close_prices = stock_data['close'].values
    
    #通过数据计算指标
    # -2是保证获取的数据是昨天的，-1就是通过今天的数据计算出来的指标
    sma_data = talib.SMA(close_prices)[-2] 
    wma_data = talib.WMA(close_prices)[-2]
    mom_data = talib.MOM(close_prices)[-2]
    
    features = []
    features.append(sma_data)
    features.append(wma_data)
    features.append(mom_data)
    
    label = False
    if close_prices[-1] > close_prices[-2]:
        label = True
    x_all.append(features)
    y_all.append(label)
    


train_X, test_X, train_y, test_y = train_test_split(x_all, y_all, test_size=0.5)
clf_forest.fit(train_X, train_y)
clf_tree.fit(train_X, train_y)

prediction_forest = clf_forest.predict(test_X)
prediction_tree = clf_tree.predict(test_X)

print(accuracy_score(test_y, prediction_forest))
print(accuracy_score(test_y, prediction_tree))

0.5247252747252747
0.5137362637362637
